In [2]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

#Read CSV file

In [3]:
stations_pd = pd.read_csv('hawaii_stations.csv')
stations_pd = stations_pd.sort_values('name')
stations_pd

,station,name,latitude,longitude,elevation
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5


In [4]:
len(stations_pd['station'])

9

In [5]:
measurements_pd = pd.read_csv('clean_measurements.csv', index_col=0)
measurements_pd.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [6]:
measurements_pd.tail()

,station,date,prcp,tobs
19543,USC00516128,2017-08-17,0.13,72
19545,USC00516128,2017-08-19,0.09,71
19547,USC00516128,2017-08-21,0.56,76
19548,USC00516128,2017-08-22,0.50,76
19549,USC00516128,2017-08-23,0.45,76


In [7]:
len(measurements_pd['station'])

18103

#Create engine in SQLite

In [8]:
engine = create_engine('sqlite:///hawaii.sqlite')

In [9]:
conn = engine.connect()

In [10]:
Base = declarative_base()

class Measurements(Base):
    __tablename__ = 'hawaii_measurements'
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return "<Measurements(%r, %r, %r, %r)>" % (self.station, self.date, self.prcp, self.tobs)

class Stations(Base):
    __tablename__ = 'hawaii_stations'
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return "<Stations(%r, %r, %r, %r, %r)>" % (self.station, self.name, self.latitude, self.longitude, self.elevation)

In [11]:
Base.metadata.create_all(engine)

In [12]:
data = measurements_pd.to_dict(orient='records')
data1 = stations_pd.to_dict(orient='records') 

In [13]:
data[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [14]:
data1[0]

{'elevation': 0.9,
 'latitude': 21.3152,
 'longitude': -157.9992,
 'name': 'HONOLULU OBSERVATORY 702.2, HI US',
 'station': 'USC00511918'}

In [15]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [16]:
measurements_table = sqlalchemy.Table('hawaii_measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('hawaii_stations', metadata, autoload=True)

In [17]:
conn.execute(measurements_table.delete())
conn.execute(measurements_table.insert(), data)

In [18]:
conn.execute(stations_table.delete())
conn.execute(stations_table.insert(), data1)

In [19]:
conn.execute("select * from hawaii_measurements limit 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64),
 (10, 'USC00519397', '2010-01-12', 0.0, 61)]

In [20]:
conn.execute("select * from hawaii_stations limit 10").fetchall()

[(1, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4),
 (5, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (6, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 (8, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (9, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5)]